<a href="https://colab.research.google.com/github/ToshikiShimizu/PyTorch/blob/master/intro_dev/chapter4/Untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch
import torch
from torch import nn, optim
from torch.utils.data import(Dataset, DataLoader, TensorDataset)
import tqdm

    100% |████████████████████████████████| 519.5MB 34kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5a3dc000 @  0x7f6360b1c2a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [3]:
!pip install torchvision

    100% |████████████████████████████████| 61kB 1.2MB/s 
    100% |████████████████████████████████| 2.0MB 13.7MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
from torchvision.datasets import FashionMNIST
from torchvision import transforms

In [5]:
fashion_mnist_train = FashionMNIST("FashionMNIST",train=True,download=True,transform=transforms.ToTensor())

Processing...
Done!


In [0]:
fashion_mnist_test = FashionMNIST("FashionMNIST",train=False,download=True,transform=transforms.ToTensor())

In [0]:
batch_size = 128
train_loader = DataLoader(fashion_mnist_train, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(fashion_mnist_test, batch_size = batch_size, shuffle = False)

In [0]:
class FlattenLayer(nn.Module):
  def forward(self, x):
    sizes = x.size()
    return x.view(sizes[0],-1)
  
conv_net = nn.Sequential(
    nn.Conv2d(1,32,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    nn.Conv2d(32,64,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.25),
    FlattenLayer()
)

In [0]:
test_input = torch.ones(1,1,28,28)
conv_output_size = conv_net(test_input).size()[-1]

mlp = nn.Sequential(
  nn.Linear(conv_output_size, 200),
  nn.ReLU(),
  nn.BatchNorm1d(200),
  nn.Dropout(0.25),
  nn.Linear(200,10),
)

net = nn.Sequential(
  conv_net,
  mlp
)

In [0]:
#device = torch.device('cpu')

In [0]:
def eval_net(net,data_loader,device="cpu"):
  net.eval()
  ys = []
  ypreds = []
  for x,y in data_loader:
    net.eval()
    ys = []
    ypreds = []
    for x, y in data_loader:
      x = x.to(device)
      y = y.to(device)
      with torch.no_grad():
        _, y_pred = net(x).max(1)
      ys.append(y)
      ypreds.append(y_pred)
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()
  


In [0]:
def train_net(net, train_loader, test_loader, optimizer_cls=optim.Adam, loss_fn=nn.CrossEntropyLoss(),n_iter=10,device="cpu"):
  train_losses = []
  train_acc = []
  val_acc = []
  optimizer = optimizer_cls(net.parameters())
  for epoch in range(n_iter):
    running_loss = 0.0
    net.train()
    n = 0
    n_acc = 0
    for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader),total=len(train_loader)):
      xx = xx.to(device)
      yy = yy.to(device)
      h = net(xx)
      loss = loss_fn(h, yy)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
      n += len(xx)
      _, y_pred = h.max(1)
      n_acc += (yy == y_pred).float().sum().item()
    train_losses.append(running_loss/i)
    train_acc.append(n_acc/n)
    val_acc.append(eval_net(net,test_loader,device))
    print (epoch, train_losses[-1],val_acc[-1],flush=True)
  

In [15]:
net.to("cuda:0")
train_net(net,train_loader,test_loader,n_iter=20,device="cuda:0")

100%|██████████| 469/469 [00:10<00:00, 43.49it/s]


0 0.3183974560955141 0.8910999894142151


100%|██████████| 469/469 [00:10<00:00, 43.70it/s]


1 0.27849046640798575 0.9020999670028687


100%|██████████| 469/469 [00:10<00:00, 43.36it/s]


...

17 0.15334892222005078 0.9217000007629395


100%|██████████| 469/469 [00:10<00:00, 43.44it/s]


18 0.1510210869451746 0.9231999516487122


100%|██████████| 469/469 [00:10<00:00, 43.53it/s]


19 0.14993680641055107 0.9192999601364136


In [0]:
net.to("cpu")
train_net(net,train_loader,test_loader,n_iter=20,device="cpu")

100%|██████████| 469/469 [01:22<00:00,  5.99it/s]


0 0.14937896391329092 0.9203000068664551


100%|██████████| 469/469 [01:24<00:00,  6.15it/s]


1 0.14377692731845582 0.91839998960495


100%|██████████| 469/469 [01:23<00:00,  6.21it/s]


...

100%|██████████| 469/469 [01:24<00:00,  6.06it/s]


4 0.13878504666857994 0.9211999773979187


100%|██████████| 469/469 [01:22<00:00,  6.04it/s]


5 0.13431807132995027 0.9228000044822693


 55%|█████▌    | 260/469 [00:46<00:38,  5.50it/s]